In [1]:
import requests
import pandas as pd
import geopandas as gpd
import json

### Part 1
Scraping School map data from MNPS site (https://mnps.maps.arcgis.com/home/item.html?id=a25c1ef058a247cfa2636a396f3aaedd#data). Table has information for geographic location (latitude and longitude) for each MNPS school. This information will be used for map generation. 

In [2]:
URL = 'https://services.arcgis.com/GBzOxI4K7WHYqcYy/arcgis/rest/services/Schools_Map/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=OBJECTID%20ASC&resultOffset=0&resultRecordCount=1000&cacheHint=true&quantizationParameters=%7B%22mode%22%3A%22edit%22%7D'

In [3]:
response = requests.get(URL)

In [4]:
School_map_data = pd.DataFrame([x['attributes'] for x in response.json()['features']])

In [5]:
School_map_data

,OBJECTID,CODE,SHORT_NAME,SCHOOL_NAME,CLUSTER,CLUS_NAME,GR_TIER,GRADES,TYPE_1,ZONE_1,...,SCHBRD_MBR,LONG_,LAT_,CLUSTER_GEO,OPTIONSCH_CURR,OPTIONSCH_NEXT,OPTID_CURR,OPTID_NEXT,TN_HOUSE,TN_SENATE
0,1,120,Jere Baxter MS,Jere Baxter Middle School,550,MAPLEWOOD,MS,6-8,DISTRICT,ZONE SCHOOL,...,Dr. Sharon Dixon Gentry,-86.750621,36.218945,MAPLEWOOD,1,0,61,0,54.0,19.0
1,2,200,Chadwell,Chadwell Elementary School,550,MAPLEWOOD,ES,PK-5,DISTRICT,ZONE SCHOOL,...,Dr. Sharon Dixon Gentry,-86.741676,36.256377,MAPLEWOOD,1,0,6,0,54.0,19.0
2,3,710,Stratton,Stratton Elementary School,452,HUNTERS LANE,ES,PK-5,DISTRICT,ZONE SCHOOL,...,Emily Masters,-86.717687,36.265610,HUNTERS LANE,0,0,0,0,54.0,19.0
3,4,545,Madison MS,Madison Middle School,452,HUNTERS LANE,MS,6-8,DISTRICT,ZONE SCHOOL,...,Emily Masters,-86.716452,36.264548,HUNTERS LANE,1,0,70,0,54.0,19.0
4,5,550,Maplewood HS,Maplewood High School,550,MAPLEWOOD,HS,9-12,DISTRICT,ZONE SCHOOL,...,Dr. Sharon Dixon Gentry,-86.750472,36.232819,MAPLEWOOD,1,0,93,0,54.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,171,285,John Early MS,John Early Middle School,632,PEARL-COHN,MS,6-8,MAGNET,ZONE SCHOOL,...,Christiane Buggs,-86.804970,36.187034,PEARL-COHN,1,0,62,0,58.0,19.0
167,172,350,Goodlettsville - Temp,Goodlettsville Elementary School,452,HUNTERS LANE,ES,PK-5,DISTRICT,ZONE SCHOOL,...,Dr. Sharon Dixon Gentry,NaN,NaN,MAPLEWOOD,0,0,0,0,50.0,20.0
168,176,179,Aventura Community School,Aventura Community School,325,GLENCLIFF,ES,K-1,CHARTER,NOT ZONE SCHOOL,...,Freda Player,NaN,NaN,None,0,0,0,0,NaN,NaN
169,178,9999,Goodlettsville,Goodlettsville Elementary School,452,HUNTERS LANE,ES,PK-4,DISTRICT,ZONE SCHOOL,...,Emily Masters,-86.710655,36.313092,HUNTERS LANE,0,0,0,0,50.0,20.0


Exporting data frame into csv file for use in R 

In [6]:
School_map_data.to_csv('School_map_dat.csv', index = True)

### Part 2
Using the American Community Survey API, obtained for each census tract, the population (B01001_001E in the detailed tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37.


In [7]:
endpoint = 'https://api.census.gov/data/2021/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2021 = requests.get(endpoint)
Population2021 = Pop_response2021.json()
Population2021_df =pd.DataFrame.from_records(Population2021)
Population2021_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 101.03, Davidson County, Tennessee",2545,47,037,010103
2,"Census Tract 101.04, Davidson County, Tennessee",3064,47,037,010104
3,"Census Tract 101.05, Davidson County, Tennessee",4895,47,037,010105
4,"Census Tract 101.06, Davidson County, Tennessee",2985,47,037,010106
...,...,...,...,...,...
170,"Census Tract 195.02, Davidson County, Tennessee",3327,47,037,019502
171,"Census Tract 195.03, Davidson County, Tennessee",3259,47,037,019503
172,"Census Tract 196, Davidson County, Tennessee",3561,47,037,019600
173,"Census Tract 9801, Davidson County, Tennessee",0,47,037,980100


In [8]:
Population2021_df = Population2021_df.rename(columns = Population2021_df.iloc[0])
Population2021_df = Population2021_df.drop(0)
Population2021_df = Population2021_df.rename(columns = { "B01001_001E" : "2021"})
Population2021_df = Population2021_df.drop(columns = ['state', 'county'])
Population2021_df 

,NAME,2021,tract
1,"Census Tract 101.03, Davidson County, Tennessee",2545,010103
2,"Census Tract 101.04, Davidson County, Tennessee",3064,010104
3,"Census Tract 101.05, Davidson County, Tennessee",4895,010105
4,"Census Tract 101.06, Davidson County, Tennessee",2985,010106
5,"Census Tract 102.01, Davidson County, Tennessee",4563,010201
...,...,...,...
170,"Census Tract 195.02, Davidson County, Tennessee",3327,019502
171,"Census Tract 195.03, Davidson County, Tennessee",3259,019503
172,"Census Tract 196, Davidson County, Tennessee",3561,019600
173,"Census Tract 9801, Davidson County, Tennessee",0,980100


In [9]:
endpoint = 'https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2020 = requests.get(endpoint)
Population2020 = Pop_response2020.json()
Population2020_df =pd.DataFrame.from_records(Population2020)
Population2020_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 190.04, Davidson County, Tennessee",4470,47,037,019004
2,"Census Tract 190.07, Davidson County, Tennessee",2904,47,037,019007
3,"Census Tract 190.08, Davidson County, Tennessee",6005,47,037,019008
4,"Census Tract 191.05, Davidson County, Tennessee",5707,47,037,019105
...,...,...,...,...,...
170,"Census Tract 189.01, Davidson County, Tennessee",3280,47,037,018901
171,"Census Tract 189.02, Davidson County, Tennessee",2704,47,037,018902
172,"Census Tract 189.04, Davidson County, Tennessee",3410,47,037,018904
173,"Census Tract 189.05, Davidson County, Tennessee",3292,47,037,018905


In [10]:
Population2020_df = Population2020_df.rename(columns = Population2020_df.iloc[0])
Population2020_df = Population2020_df.drop(0)
Population2020_df = Population2020_df.rename(columns = { "B01001_001E" : "2020"})
Population2020_df= Population2020_df.drop(columns = ['state', 'county'])
Population2020_df 

,NAME,2020,tract
1,"Census Tract 190.04, Davidson County, Tennessee",4470,019004
2,"Census Tract 190.07, Davidson County, Tennessee",2904,019007
3,"Census Tract 190.08, Davidson County, Tennessee",6005,019008
4,"Census Tract 191.05, Davidson County, Tennessee",5707,019105
5,"Census Tract 191.06, Davidson County, Tennessee",4336,019106
...,...,...,...
170,"Census Tract 189.01, Davidson County, Tennessee",3280,018901
171,"Census Tract 189.02, Davidson County, Tennessee",2704,018902
172,"Census Tract 189.04, Davidson County, Tennessee",3410,018904
173,"Census Tract 189.05, Davidson County, Tennessee",3292,018905


In [11]:
endpoint = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2019 = requests.get(endpoint)
Population2019 = Pop_response2019.json()
Population2019_df =pd.DataFrame.from_records(Population2019)
Population2019_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 156.31, Davidson County, Tennessee",12176,47,037,015631
2,"Census Tract 158.04, Davidson County, Tennessee",4098,47,037,015804
3,"Census Tract 177.01, Davidson County, Tennessee",2466,47,037,017701
4,"Census Tract 177.02, Davidson County, Tennessee",5210,47,037,017702
...,...,...,...,...,...
157,"Census Tract 156.13, Davidson County, Tennessee",5252,47,037,015613
158,"Census Tract 156.20, Davidson County, Tennessee",7262,47,037,015620
159,"Census Tract 161, Davidson County, Tennessee",2345,47,037,016100
160,"Census Tract 114, Davidson County, Tennessee",4617,47,037,011400


In [12]:
Population2019_df = Population2019_df.rename(columns = Population2019_df.iloc[0])
Population2019_df = Population2019_df.drop(0)
Population2019_df = Population2019_df.rename(columns = { "B01001_001E" : "2019"})
Population2019_df = Population2019_df.drop(columns = ['state', 'county'])
Population2019_df 

,NAME,2019,tract
1,"Census Tract 156.31, Davidson County, Tennessee",12176,015631
2,"Census Tract 158.04, Davidson County, Tennessee",4098,015804
3,"Census Tract 177.01, Davidson County, Tennessee",2466,017701
4,"Census Tract 177.02, Davidson County, Tennessee",5210,017702
5,"Census Tract 183.01, Davidson County, Tennessee",8254,018301
...,...,...,...
157,"Census Tract 156.13, Davidson County, Tennessee",5252,015613
158,"Census Tract 156.20, Davidson County, Tennessee",7262,015620
159,"Census Tract 161, Davidson County, Tennessee",2345,016100
160,"Census Tract 114, Davidson County, Tennessee",4617,011400


In [13]:
endpoint = 'https://api.census.gov/data/2018/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2018 = requests.get(endpoint)
Population2018 = Pop_response2018.json()
Population2018_df = pd.DataFrame.from_records(Population2018)
Population2018_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 133, Davidson County, Tennessee",3575,47,037,013300
2,"Census Tract 172, Davidson County, Tennessee",1455,47,037,017200
3,"Census Tract 184.08, Davidson County, Tennessee",3733,47,037,018408
4,"Census Tract 114, Davidson County, Tennessee",4650,47,037,011400
...,...,...,...,...,...
157,"Census Tract 109.03, Davidson County, Tennessee",5564,47,037,010903
158,"Census Tract 139, Davidson County, Tennessee",1720,47,037,013900
159,"Census Tract 189.01, Davidson County, Tennessee",3194,47,037,018901
160,"Census Tract 178, Davidson County, Tennessee",5706,47,037,017800


In [14]:
Population2018_df = Population2018_df.rename(columns = Population2018_df.iloc[0])
Population2018_df = Population2018_df.drop(0)
Population2018_df = Population2018_df.rename(columns = { "B01001_001E" : "2018"})
Population2018_df = Population2018_df.drop(columns = ['state', 'county'])
Population2018_df 

,NAME,2018,tract
1,"Census Tract 133, Davidson County, Tennessee",3575,013300
2,"Census Tract 172, Davidson County, Tennessee",1455,017200
3,"Census Tract 184.08, Davidson County, Tennessee",3733,018408
4,"Census Tract 114, Davidson County, Tennessee",4650,011400
5,"Census Tract 138, Davidson County, Tennessee",1963,013800
...,...,...,...
157,"Census Tract 109.03, Davidson County, Tennessee",5564,010903
158,"Census Tract 139, Davidson County, Tennessee",1720,013900
159,"Census Tract 189.01, Davidson County, Tennessee",3194,018901
160,"Census Tract 178, Davidson County, Tennessee",5706,017800


In [15]:
endpoint = 'https://api.census.gov/data/2017/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2017 = requests.get(endpoint)
Population2017 = Pop_response2017.json()
Population2017_df = pd.DataFrame.from_records(Population2017)
Population2017_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 134, Davidson County, Tennessee",4371,47,037,013400
2,"Census Tract 173, Davidson County, Tennessee",3156,47,037,017300
3,"Census Tract 179.01, Davidson County, Tennessee",4396,47,037,017901
4,"Census Tract 118, Davidson County, Tennessee",2722,47,037,011800
...,...,...,...,...,...
157,"Census Tract 178, Davidson County, Tennessee",5498,47,037,017800
158,"Census Tract 159, Davidson County, Tennessee",2896,47,037,015900
159,"Census Tract 133, Davidson County, Tennessee",3557,47,037,013300
160,"Census Tract 172, Davidson County, Tennessee",1523,47,037,017200


In [16]:
Population2017_df = Population2017_df.rename(columns = Population2017_df.iloc[0])
Population2017_df = Population2017_df.drop(0)
Population2017_df = Population2017_df.rename(columns = { "B01001_001E" : "2017"})
Population2017_df = Population2017_df.drop(columns = ['state', 'county'])
Population2017_df 

,NAME,2017,tract
1,"Census Tract 134, Davidson County, Tennessee",4371,013400
2,"Census Tract 173, Davidson County, Tennessee",3156,017300
3,"Census Tract 179.01, Davidson County, Tennessee",4396,017901
4,"Census Tract 118, Davidson County, Tennessee",2722,011800
5,"Census Tract 109.01, Davidson County, Tennessee",3256,010901
...,...,...,...
157,"Census Tract 178, Davidson County, Tennessee",5498,017800
158,"Census Tract 159, Davidson County, Tennessee",2896,015900
159,"Census Tract 133, Davidson County, Tennessee",3557,013300
160,"Census Tract 172, Davidson County, Tennessee",1523,017200


In [17]:
endpoint = 'https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2016 = requests.get(endpoint)
Population2016 = Pop_response2016.json()
Population2016_df = pd.DataFrame.from_records(Population2016)
Population2016_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 166, Davidson County, Tennessee",2747,47,037,016600
2,"Census Tract 167, Davidson County, Tennessee",5106,47,037,016700
3,"Census Tract 169, Davidson County, Tennessee",4909,47,037,016900
4,"Census Tract 179.02, Davidson County, Tennessee",4792,47,037,017902
...,...,...,...,...,...
157,"Census Tract 154.04, Davidson County, Tennessee",2754,47,037,015404
158,"Census Tract 130, Davidson County, Tennessee",1117,47,037,013000
159,"Census Tract 156.17, Davidson County, Tennessee",2379,47,037,015617
160,"Census Tract 156.19, Davidson County, Tennessee",5218,47,037,015619


In [18]:
Population2016_df = Population2016_df.rename(columns = Population2016_df.iloc[0])
Population2016_df = Population2016_df.drop(0)
Population2016_df = Population2016_df.rename(columns = { "B01001_001E" : "2016"})
Population2016_df = Population2016_df.drop(columns = ['state', 'county'])
Population2016_df 

,NAME,2016,tract
1,"Census Tract 166, Davidson County, Tennessee",2747,016600
2,"Census Tract 167, Davidson County, Tennessee",5106,016700
3,"Census Tract 169, Davidson County, Tennessee",4909,016900
4,"Census Tract 179.02, Davidson County, Tennessee",4792,017902
5,"Census Tract 107.02, Davidson County, Tennessee",3363,010702
...,...,...,...
157,"Census Tract 154.04, Davidson County, Tennessee",2754,015404
158,"Census Tract 130, Davidson County, Tennessee",1117,013000
159,"Census Tract 156.17, Davidson County, Tennessee",2379,015617
160,"Census Tract 156.19, Davidson County, Tennessee",5218,015619


In [19]:
endpoint = 'https://api.census.gov/data/2015/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2015 = requests.get(endpoint)
Population2015 = Pop_response2015.json()
Population2015_df = pd.DataFrame.from_records(Population2015)
Population2015_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 184.05, Davidson County, Tennessee",4719,47,037,018405
2,"Census Tract 184.07, Davidson County, Tennessee",4384,47,037,018407
3,"Census Tract 186.01, Davidson County, Tennessee",2323,47,037,018601
4,"Census Tract 191.08, Davidson County, Tennessee",3362,47,037,019108
...,...,...,...,...,...
157,"Census Tract 102.01, Davidson County, Tennessee",4697,47,037,010201
158,"Census Tract 106.02, Davidson County, Tennessee",3905,47,037,010602
159,"Census Tract 136.02, Davidson County, Tennessee",1956,47,037,013602
160,"Census Tract 156.24, Davidson County, Tennessee",5101,47,037,015624


In [20]:
Population2015_df = Population2015_df.rename(columns = Population2015_df.iloc[0])
Population2015_df = Population2015_df.drop(0)
Population2015_df = Population2015_df.rename(columns = { "B01001_001E" : "2015"})
Population2015_df = Population2015_df.drop(columns = ['state', 'county'])
Population2015_df 

,NAME,2015,tract
1,"Census Tract 184.05, Davidson County, Tennessee",4719,018405
2,"Census Tract 184.07, Davidson County, Tennessee",4384,018407
3,"Census Tract 186.01, Davidson County, Tennessee",2323,018601
4,"Census Tract 191.08, Davidson County, Tennessee",3362,019108
5,"Census Tract 182.01, Davidson County, Tennessee",2875,018201
...,...,...,...
157,"Census Tract 102.01, Davidson County, Tennessee",4697,010201
158,"Census Tract 106.02, Davidson County, Tennessee",3905,010602
159,"Census Tract 136.02, Davidson County, Tennessee",1956,013602
160,"Census Tract 156.24, Davidson County, Tennessee",5101,015624


In [21]:
endpoint = 'https://api.census.gov/data/2014/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2014 = requests.get(endpoint)
Population2014 = Pop_response2014.json()
Population2014_df = pd.DataFrame.from_records(Population2014)
Population2014_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 103.02, Davidson County, Tennessee",1925,47,037,010302
2,"Census Tract 112, Davidson County, Tennessee",3895,47,037,011200
3,"Census Tract 104.02, Davidson County, Tennessee",5411,47,037,010402
4,"Census Tract 105.02, Davidson County, Tennessee",5063,47,037,010502
...,...,...,...,...,...
157,"Census Tract 154.04, Davidson County, Tennessee",3238,47,037,015404
158,"Census Tract 130, Davidson County, Tennessee",1664,47,037,013000
159,"Census Tract 156.17, Davidson County, Tennessee",2459,47,037,015617
160,"Census Tract 156.19, Davidson County, Tennessee",4782,47,037,015619


In [22]:
Population2014_df = Population2014_df.rename(columns = Population2014_df.iloc[0])
Population2014_df = Population2014_df.drop(0)
Population2014_df = Population2014_df.rename(columns = { "B01001_001E" : "2014"})
Population2014_df = Population2014_df.drop(columns = ['state', 'county'])
Population2014_df 

,NAME,2014,tract
1,"Census Tract 103.02, Davidson County, Tennessee",1925,010302
2,"Census Tract 112, Davidson County, Tennessee",3895,011200
3,"Census Tract 104.02, Davidson County, Tennessee",5411,010402
4,"Census Tract 105.02, Davidson County, Tennessee",5063,010502
5,"Census Tract 109.04, Davidson County, Tennessee",2784,010904
...,...,...,...
157,"Census Tract 154.04, Davidson County, Tennessee",3238,015404
158,"Census Tract 130, Davidson County, Tennessee",1664,013000
159,"Census Tract 156.17, Davidson County, Tennessee",2459,015617
160,"Census Tract 156.19, Davidson County, Tennessee",4782,015619


In [23]:
endpoint = 'https://api.census.gov/data/2013/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2013 = requests.get(endpoint)
Population2013 = Pop_response2013.json()
Population2013_df = pd.DataFrame.from_records(Population2013)
Population2013_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 184.05, Davidson County, Tennessee",4883,47,037,018405
2,"Census Tract 184.07, Davidson County, Tennessee",4347,47,037,018407
3,"Census Tract 186.01, Davidson County, Tennessee",2162,47,037,018601
4,"Census Tract 191.08, Davidson County, Tennessee",3453,47,037,019108
...,...,...,...,...,...
157,"Census Tract 194, Davidson County, Tennessee",3464,47,037,019400
158,"Census Tract 195, Davidson County, Tennessee",5282,47,037,019500
159,"Census Tract 196, Davidson County, Tennessee",3757,47,037,019600
160,"Census Tract 9801, Davidson County, Tennessee",0,47,037,980100


In [24]:
Population2013_df = Population2013_df.rename(columns = Population2013_df.iloc[0])
Population2013_df = Population2013_df.drop(0)
Population2013_df = Population2013_df.rename(columns = { "B01001_001E" : "2013"})
Population2013_df = Population2013_df.drop(columns = ['state', 'county'])
Population2013_df 

,NAME,2013,tract
1,"Census Tract 184.05, Davidson County, Tennessee",4883,018405
2,"Census Tract 184.07, Davidson County, Tennessee",4347,018407
3,"Census Tract 186.01, Davidson County, Tennessee",2162,018601
4,"Census Tract 191.08, Davidson County, Tennessee",3453,019108
5,"Census Tract 182.01, Davidson County, Tennessee",2909,018201
...,...,...,...
157,"Census Tract 194, Davidson County, Tennessee",3464,019400
158,"Census Tract 195, Davidson County, Tennessee",5282,019500
159,"Census Tract 196, Davidson County, Tennessee",3757,019600
160,"Census Tract 9801, Davidson County, Tennessee",0,980100


In [25]:
endpoint = 'https://api.census.gov/data/2012/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2012 = requests.get(endpoint)
Population2012 = Pop_response2012.json()
Population2012_df = pd.DataFrame.from_records(Population2012)
Population2012_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 188.03, Davidson County, Tennessee",3354,47,037,018803
2,"Census Tract 188.04, Davidson County, Tennessee",5179,47,037,018804
3,"Census Tract 189.04, Davidson County, Tennessee",3867,47,037,018904
4,"Census Tract 189.05, Davidson County, Tennessee",2984,47,037,018905
...,...,...,...,...,...
157,"Census Tract 117, Davidson County, Tennessee",5600,47,037,011700
158,"Census Tract 156.13, Davidson County, Tennessee",5950,47,037,015613
159,"Census Tract 156.20, Davidson County, Tennessee",6977,47,037,015620
160,"Census Tract 161, Davidson County, Tennessee",1941,47,037,016100


In [26]:
Population2012_df = Population2012_df.rename(columns = Population2012_df.iloc[0])
Population2012_df = Population2012_df.drop(0)
Population2012_df = Population2012_df.rename(columns = { "B01001_001E" : "2012"})
Population2012_df = Population2012_df.drop(columns = ['state', 'county'])
Population2012_df 

,NAME,2012,tract
1,"Census Tract 188.03, Davidson County, Tennessee",3354,018803
2,"Census Tract 188.04, Davidson County, Tennessee",5179,018804
3,"Census Tract 189.04, Davidson County, Tennessee",3867,018904
4,"Census Tract 189.05, Davidson County, Tennessee",2984,018905
5,"Census Tract 190.05, Davidson County, Tennessee",3253,019005
...,...,...,...
157,"Census Tract 117, Davidson County, Tennessee",5600,011700
158,"Census Tract 156.13, Davidson County, Tennessee",5950,015613
159,"Census Tract 156.20, Davidson County, Tennessee",6977,015620
160,"Census Tract 161, Davidson County, Tennessee",1941,016100


In [27]:
endpoint = 'https://api.census.gov/data/2011/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2011 = requests.get(endpoint)
Population2011 = Pop_response2011.json()
Population2011_df = pd.DataFrame.from_records(Population2011)
Population2011_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 110.02, Davidson County, Tennessee",2648,47,037,011002
2,"Census Tract 143, Davidson County, Tennessee",2002,47,037,014300
3,"Census Tract 156.31, Davidson County, Tennessee",8025,47,037,015631
4,"Census Tract 158.04, Davidson County, Tennessee",4219,47,037,015804
...,...,...,...,...,...
157,"Census Tract 102.01, Davidson County, Tennessee",4089,47,037,010201
158,"Census Tract 106.02, Davidson County, Tennessee",3133,47,037,010602
159,"Census Tract 136.02, Davidson County, Tennessee",2540,47,037,013602
160,"Census Tract 156.24, Davidson County, Tennessee",4947,47,037,015624


In [28]:
Population2011_df = Population2011_df.rename(columns = Population2011_df.iloc[0])
Population2011_df = Population2011_df.drop(0)
Population2011_df = Population2011_df.rename(columns = { "B01001_001E" : "2011"})
Population2011_df = Population2011_df.drop(columns = ['state', 'county'])
Population2011_df 

,NAME,2011,tract
1,"Census Tract 110.02, Davidson County, Tennessee",2648,011002
2,"Census Tract 143, Davidson County, Tennessee",2002,014300
3,"Census Tract 156.31, Davidson County, Tennessee",8025,015631
4,"Census Tract 158.04, Davidson County, Tennessee",4219,015804
5,"Census Tract 177.01, Davidson County, Tennessee",1983,017701
...,...,...,...
157,"Census Tract 102.01, Davidson County, Tennessee",4089,010201
158,"Census Tract 106.02, Davidson County, Tennessee",3133,010602
159,"Census Tract 136.02, Davidson County, Tennessee",2540,013602
160,"Census Tract 156.24, Davidson County, Tennessee",4947,015624


In [29]:
endpoint= 'https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2010 = requests.get(endpoint)
Population2010 = Pop_response2010.json()
Population2010_df = pd.DataFrame.from_records(Population2010)
Population2010_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 101.03, Davidson County, Tennessee",2176,47,037,010103
2,"Census Tract 101.04, Davidson County, Tennessee",3475,47,037,010104
3,"Census Tract 101.05, Davidson County, Tennessee",4322,47,037,010105
4,"Census Tract 101.06, Davidson County, Tennessee",2300,47,037,010106
...,...,...,...,...,...
157,"Census Tract 194, Davidson County, Tennessee",3114,47,037,019400
158,"Census Tract 195, Davidson County, Tennessee",4450,47,037,019500
159,"Census Tract 196, Davidson County, Tennessee",3097,47,037,019600
160,"Census Tract 9801, Davidson County, Tennessee",0,47,037,980100


In [30]:
Population2010_df = Population2010_df.rename(columns = Population2010_df.iloc[0])
Population2010_df = Population2010_df.drop(0)
Population2010_df = Population2010_df.rename(columns = { "B01001_001E" : "2010"})
Population2010_df = Population2010_df.drop(columns = ['state', 'county'])
Population2010_df 

,NAME,2010,tract
1,"Census Tract 101.03, Davidson County, Tennessee",2176,010103
2,"Census Tract 101.04, Davidson County, Tennessee",3475,010104
3,"Census Tract 101.05, Davidson County, Tennessee",4322,010105
4,"Census Tract 101.06, Davidson County, Tennessee",2300,010106
5,"Census Tract 102.01, Davidson County, Tennessee",3987,010201
...,...,...,...
157,"Census Tract 194, Davidson County, Tennessee",3114,019400
158,"Census Tract 195, Davidson County, Tennessee",4450,019500
159,"Census Tract 196, Davidson County, Tennessee",3097,019600
160,"Census Tract 9801, Davidson County, Tennessee",0,980100


In [31]:
endpoint= 'https://api.census.gov/data/2009/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2009 = requests.get(endpoint)
Population2009= Pop_response2009.json()
Population2009_df=pd.DataFrame.from_records(Population2009)
Population2009_df

,0,1,2,3,4
0,NAME,B01001_001E,state,county,tract
1,"Census Tract 117, Davidson County, Tennessee",5156,47,037,011700
2,"Census Tract 118, Davidson County, Tennessee",2776,47,037,011800
3,"Census Tract 119, Davidson County, Tennessee",1869,47,037,011900
4,"Census Tract 120, Davidson County, Tennessee",868,47,037,012000
...,...,...,...,...,...
140,"Census Tract 112, Davidson County, Tennessee",4310,47,037,011200
141,"Census Tract 113, Davidson County, Tennessee",5724,47,037,011300
142,"Census Tract 114, Davidson County, Tennessee",5284,47,037,011400
143,"Census Tract 115, Davidson County, Tennessee",4320,47,037,011500


In [32]:
Population2009_df = Population2009_df.rename(columns = Population2009_df.iloc[0])
Population2009_df = Population2009_df.drop(0)
Population2009_df = Population2009_df.rename(columns = { "B01001_001E" : "2009"})
Population2009_df = Population2009_df.drop(columns = ['state', 'county'])
Population2009_df 

,NAME,2009,tract
1,"Census Tract 117, Davidson County, Tennessee",5156,011700
2,"Census Tract 118, Davidson County, Tennessee",2776,011800
3,"Census Tract 119, Davidson County, Tennessee",1869,011900
4,"Census Tract 120, Davidson County, Tennessee",868,012000
5,"Census Tract 121, Davidson County, Tennessee",2724,012100
...,...,...,...
140,"Census Tract 112, Davidson County, Tennessee",4310,011200
141,"Census Tract 113, Davidson County, Tennessee",5724,011300
142,"Census Tract 114, Davidson County, Tennessee",5284,011400
143,"Census Tract 115, Davidson County, Tennessee",4320,011500


In [33]:
Population = pd.merge(left = Population2021_df, 
         right = Population2020_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283
...,...,...,...,...
169,"Census Tract 195.02, Davidson County, Tennessee",3327,019502,3115
170,"Census Tract 195.03, Davidson County, Tennessee",3259,019503,3508
171,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828
172,"Census Tract 9801, Davidson County, Tennessee",0,980100,0


In [34]:
Population = pd.merge(left = Population, 
         right = Population2019_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020,2019
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411,2533
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002,2955
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839,4435
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948,3397
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283,4157
...,...,...,...,...,...
138,"Census Tract 192, Davidson County, Tennessee",3776,019200,3853,3897
139,"Census Tract 193, Davidson County, Tennessee",3977,019300,3498,3428
140,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828,3843
141,"Census Tract 9801, Davidson County, Tennessee",0,980100,0,0


In [35]:
Population = pd.merge(left = Population, 
         right = Population2018_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020,2019,2018
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411,2533,2572
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002,2955,2934
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839,4435,4132
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948,3397,3282
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283,4157,4677
...,...,...,...,...,...,...
138,"Census Tract 192, Davidson County, Tennessee",3776,019200,3853,3897,3741
139,"Census Tract 193, Davidson County, Tennessee",3977,019300,3498,3428,3415
140,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828,3843,4164
141,"Census Tract 9801, Davidson County, Tennessee",0,980100,0,0,0


In [36]:
Population = pd.merge(left = Population, 
         right = Population2017_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020,2019,2018,2017
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411,2533,2572,2539
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002,2955,2934,2907
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839,4435,4132,4254
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948,3397,3282,3335
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283,4157,4677,4686
...,...,...,...,...,...,...,...
138,"Census Tract 192, Davidson County, Tennessee",3776,019200,3853,3897,3741,3781
139,"Census Tract 193, Davidson County, Tennessee",3977,019300,3498,3428,3415,3244
140,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828,3843,4164,4259
141,"Census Tract 9801, Davidson County, Tennessee",0,980100,0,0,0,0


In [37]:
Population = pd.merge(left = Population, 
         right = Population2016_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020,2019,2018,2017,2016
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411,2533,2572,2539,2627
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002,2955,2934,2907,2821
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839,4435,4132,4254,4257
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948,3397,3282,3335,3257
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283,4157,4677,4686,4638
...,...,...,...,...,...,...,...,...
138,"Census Tract 192, Davidson County, Tennessee",3776,019200,3853,3897,3741,3781,3782
139,"Census Tract 193, Davidson County, Tennessee",3977,019300,3498,3428,3415,3244,3163
140,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828,3843,4164,4259,4216
141,"Census Tract 9801, Davidson County, Tennessee",0,980100,0,0,0,0,0


In [38]:
Population = pd.merge(left = Population, 
         right = Population2015_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020,2019,2018,2017,2016,2015
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411,2533,2572,2539,2627,2551
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002,2955,2934,2907,2821,3000
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839,4435,4132,4254,4257,4453
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948,3397,3282,3335,3257,3175
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283,4157,4677,4686,4638,4697
...,...,...,...,...,...,...,...,...,...
138,"Census Tract 192, Davidson County, Tennessee",3776,019200,3853,3897,3741,3781,3782,3644
139,"Census Tract 193, Davidson County, Tennessee",3977,019300,3498,3428,3415,3244,3163,3370
140,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828,3843,4164,4259,4216,3952
141,"Census Tract 9801, Davidson County, Tennessee",0,980100,0,0,0,0,0,0


In [39]:
Population = pd.merge(left = Population, 
         right = Population2014_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020,2019,2018,2017,2016,2015,2014
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411,2533,2572,2539,2627,2551,2643
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002,2955,2934,2907,2821,3000,3142
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839,4435,4132,4254,4257,4453,4199
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948,3397,3282,3335,3257,3175,3153
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283,4157,4677,4686,4638,4697,4596
...,...,...,...,...,...,...,...,...,...,...
138,"Census Tract 192, Davidson County, Tennessee",3776,019200,3853,3897,3741,3781,3782,3644,3425
139,"Census Tract 193, Davidson County, Tennessee",3977,019300,3498,3428,3415,3244,3163,3370,3319
140,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828,3843,4164,4259,4216,3952,3657
141,"Census Tract 9801, Davidson County, Tennessee",0,980100,0,0,0,0,0,0,0


In [40]:
Population = pd.merge(left = Population, 
         right = Population2013_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020,2019,2018,2017,2016,2015,2014,2013
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411,2533,2572,2539,2627,2551,2643,2431
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002,2955,2934,2907,2821,3000,3142,3236
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839,4435,4132,4254,4257,4453,4199,4200
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948,3397,3282,3335,3257,3175,3153,3124
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283,4157,4677,4686,4638,4697,4596,4500
...,...,...,...,...,...,...,...,...,...,...,...
138,"Census Tract 192, Davidson County, Tennessee",3776,019200,3853,3897,3741,3781,3782,3644,3425,3202
139,"Census Tract 193, Davidson County, Tennessee",3977,019300,3498,3428,3415,3244,3163,3370,3319,3071
140,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828,3843,4164,4259,4216,3952,3657,3757
141,"Census Tract 9801, Davidson County, Tennessee",0,980100,0,0,0,0,0,0,0,0


In [41]:
Population = pd.merge(left = Population, 
         right = Population2012_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020,2019,2018,2017,2016,2015,2014,2013,2012
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411,2533,2572,2539,2627,2551,2643,2431,2308
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002,2955,2934,2907,2821,3000,3142,3236,3460
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839,4435,4132,4254,4257,4453,4199,4200,4255
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948,3397,3282,3335,3257,3175,3153,3124,2768
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283,4157,4677,4686,4638,4697,4596,4500,4398
...,...,...,...,...,...,...,...,...,...,...,...,...
138,"Census Tract 192, Davidson County, Tennessee",3776,019200,3853,3897,3741,3781,3782,3644,3425,3202,3246
139,"Census Tract 193, Davidson County, Tennessee",3977,019300,3498,3428,3415,3244,3163,3370,3319,3071,3102
140,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828,3843,4164,4259,4216,3952,3657,3757,3607
141,"Census Tract 9801, Davidson County, Tennessee",0,980100,0,0,0,0,0,0,0,0,0


In [42]:
Population = pd.merge(left = Population, 
         right = Population2011_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411,2533,2572,2539,2627,2551,2643,2431,2308,2257
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002,2955,2934,2907,2821,3000,3142,3236,3460,3431
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839,4435,4132,4254,4257,4453,4199,4200,4255,4664
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948,3397,3282,3335,3257,3175,3153,3124,2768,2503
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283,4157,4677,4686,4638,4697,4596,4500,4398,4089
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,"Census Tract 192, Davidson County, Tennessee",3776,019200,3853,3897,3741,3781,3782,3644,3425,3202,3246,3281
139,"Census Tract 193, Davidson County, Tennessee",3977,019300,3498,3428,3415,3244,3163,3370,3319,3071,3102,3120
140,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828,3843,4164,4259,4216,3952,3657,3757,3607,3397
141,"Census Tract 9801, Davidson County, Tennessee",0,980100,0,0,0,0,0,0,0,0,0,0


In [43]:
Population = pd.merge(left = Population, 
         right = Population2010_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

,NAME,2021,tract,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
0,"Census Tract 101.03, Davidson County, Tennessee",2545,010103,2411,2533,2572,2539,2627,2551,2643,2431,2308,2257,2176
1,"Census Tract 101.04, Davidson County, Tennessee",3064,010104,3002,2955,2934,2907,2821,3000,3142,3236,3460,3431,3475
2,"Census Tract 101.05, Davidson County, Tennessee",4895,010105,4839,4435,4132,4254,4257,4453,4199,4200,4255,4664,4322
3,"Census Tract 101.06, Davidson County, Tennessee",2985,010106,2948,3397,3282,3335,3257,3175,3153,3124,2768,2503,2300
4,"Census Tract 102.01, Davidson County, Tennessee",4563,010201,4283,4157,4677,4686,4638,4697,4596,4500,4398,4089,3987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,"Census Tract 192, Davidson County, Tennessee",3776,019200,3853,3897,3741,3781,3782,3644,3425,3202,3246,3281,3223
139,"Census Tract 193, Davidson County, Tennessee",3977,019300,3498,3428,3415,3244,3163,3370,3319,3071,3102,3120,3064
140,"Census Tract 196, Davidson County, Tennessee",3561,019600,3828,3843,4164,4259,4216,3952,3657,3757,3607,3397,3097
141,"Census Tract 9801, Davidson County, Tennessee",0,980100,0,0,0,0,0,0,0,0,0,0,0


In [44]:
Population.to_csv('population_census_tract.csv', index = True)